
# Notebook base: Ruta óptima de surtido (TSP/VRP light) en un supermercado

**Objetivo:** Construir un ejemplo mínimo reproducible para:
1) modelar una tienda como un grafo,
2) calcular distancias entre puntos relevantes (inicio, ítems, fin),
3) generar una ruta inicial con **Nearest Neighbor**,
4) mejorar la ruta con **2-opt**,
5) visualizar la ruta resultante.

> Tip: Este notebook no requiere conexión a internet. Solo usa `networkx` y `matplotlib`.


In [ ]:

# === Imports ===
import math
import itertools
from heapq import heappush, heappop

try:
    import networkx as nx
except ImportError as e:
    raise RuntimeError("Este notebook requiere 'networkx'. Instálalo con: pip install networkx")

import matplotlib.pyplot as plt



## 1. Construcción del grafo de la tienda

- Usaremos una **rejilla 5×4** (5 columnas × 4 filas) como plano simplificado de una tienda.
- Cada intersección es un nodo: `(x, y)` con `x ∈ {0..4}`, `y ∈ {0..3}`.
- Conectaremos nodos adyacentes (arriba/abajo/izquierda/derecha) con peso 1 (tiempo/distancia).
- Agregaremos **dos pasillos unidireccionales** como ejemplo de restricción operativa.


In [ ]:

# === Grafo base (rejilla 5x4) ===
G = nx.DiGraph()
positions = {}  # para dibujar

cols, rows = 5, 4  # 5 columnas (x=0..4), 4 filas (y=0..3)

for x in range(cols):
    for y in range(rows):
        node = (x, y)
        positions[node] = (x, -y)  # invertir y para dibujar "hacia abajo"
        G.add_node(node)

# Conectar vecinos con aristas bidireccionales de peso 1
def add_undirected(u, v, w=1):
    G.add_edge(u, v, weight=w)
    G.add_edge(v, u, weight=w)

for x in range(cols):
    for y in range(rows):
        u = (x, y)
        # derecha
        if x + 1 < cols:
            v = (x+1, y)
            add_undirected(u, v, 1)
        # abajo
        if y + 1 < rows:
            v = (x, y+1)
            add_undirected(u, v, 1)

# Ejemplos de pasillos unidireccionales (restricciones)
# Solo permitir avanzar de (1,1)->(2,1)->(3,1), no al revés
if G.has_edge((2,1),(1,1)): G.remove_edge((2,1),(1,1))
if G.has_edge((3,1),(2,1)): G.remove_edge((3,1),(2,1))

# Solo permitir avanzar de (3,2)->(2,2)->(1,2), no al revés
if G.has_edge((1,2),(2,2)): G.remove_edge((1,2),(2,2))
if G.has_edge((2,2),(3,2)): G.remove_edge((2,2),(3,2))

# Nodos especiales
start = (0, 0)   # entrada/almacén
end   = (4, 3)   # empaque/salida

# Ítems del pedido (elige 5-7 para ejemplo)
items = [(4,0), (1,3), (0,2), (3,2), (2,1), (4,2)]

print("Nodos del pedido:", items)
print("Total de nodos en G:", G.number_of_nodes(), "| Aristas:", G.number_of_edges())



## 2. Distancias más cortas entre puntos relevantes

Calcularemos la **longitud de camino más corto** (por tiempo/distancia) entre todos los puntos de interés:
- `start` → cada ítem,
- ítem → ítem,
- ítem → `end`.

Usamos el peso `weight` de las aristas.


In [ ]:

def shortest_time_length(graph, source, target):
    """Devuelve el tiempo/distancia mínima entre source y target usando el peso 'weight'."""
    return nx.shortest_path_length(graph, source=source, target=target, weight='weight')

def shortest_path_nodes(graph, source, target):
    """Devuelve la lista de nodos del camino más corto (útil para visualizar rutas)."""
    return nx.shortest_path(graph, source=source, target=target, weight='weight')

# Construimos el conjunto P = start, items, end
P = [start] + items + [end]

# Matriz D: dict anidado con tiempos mínimos entre puntos de P
D = {a: {} for a in P}
for a in P:
    for b in P:
        if a == b:
            continue
        D[a][b] = shortest_time_length(G, a, b)

# Mostrar una muestra de distancias
print("Ejemplo D[start][items[0]] =", D[start][items[0]])
print("Ejemplo D[items[0]][end]  =", D[items[0]][end])



## 3. Construcción de ruta inicial con **Nearest Neighbor** (paso a paso)

Estrategia:
1. Iniciar en `start`.
2. Elegir el ítem **no visitado** más cercano (según `D`) y avanzar.
3. Repetir hasta visitar todos los ítems.
4. Ir a `end`.

Mostramos el **log de decisiones** y el costo inicial.


In [ ]:

def route_cost(order, D, start, end):
    assert len(order) > 0, "La orden no puede ser vacía"
    cost = D[start][order[0]]
    for a, b in zip(order, order[1:]):
        cost += D[a][b]
    cost += D[order[-1]][end]
    return cost

def nearest_neighbor(order_nodes, D, start, end):
    unvisited = set(order_nodes)
    path = []
    curr = start
    log = []
    while unvisited:
        nxt = min(unvisited, key=lambda x: D[curr][x])
        log.append(f"{curr} -> {nxt} (costo Δ={D[curr][nxt]})")
        path.append(nxt)
        unvisited.remove(nxt)
        curr = nxt
    init_cost = route_cost(path, D, start, end)
    return path, init_cost, log

nn_path, nn_cost, nn_log = nearest_neighbor(items, D, start, end)
print("== Log Nearest Neighbor ==")
for line in nn_log:
    print(line)
print("\nRuta inicial:", nn_path)
print("Costo inicial:", nn_cost)



## 4. Mejora local con **2-opt**

**2-opt** intenta mejorar la ruta revirtiendo segmentos `(i:k)` y aceptando el cambio si reduce el costo.
Mostramos las mejoras aceptadas.


In [ ]:

def two_opt(path, D, start, end):
    def route_total(p):
        return route_cost(p, D, start, end)

    best = path[:]
    best_cost = route_total(best)
    improved = True
    accepted_moves = []

    while improved:
        improved = False
        for i in range(1, len(best)-1):
            for k in range(i+1, len(best)):
                candidate = best[:i] + best[i:k][::-1] + best[k:]
                c = route_total(candidate)
                if c < best_cost:
                    accepted_moves.append((i, k, best_cost, c, candidate[:]))
                    best, best_cost = candidate, c
                    improved = True
                    break
            if improved:
                break
    return best, best_cost, accepted_moves

opt_path, opt_cost, moves = two_opt(nn_path, D, start, end)

print("== Movimientos 2-opt aceptados ==")
if not moves:
    print("No hubo mejoras 2-opt (la ruta inicial ya era buena para este caso).")
else:
    for (i,k,old,new,cand) in moves:
        print(f"[i={i}, k={k}] costo {old} -> {new} | ruta: {cand}")

print("\nRuta optimizada:", opt_path)
print("Costo optimizado:", opt_cost)



## 5. Visualización del grafo y la ruta final

- Se dibuja el grafo en coordenadas de la rejilla.
- Se resaltan (grosor de línea) los tramos de la ruta óptima **según caminos más cortos** entre cada par consecutivo en `start → ...items... → end`.
> Nota: No se especifican colores manualmente; se utilizan estilos por defecto.


In [ ]:

def edges_in_shortest_path(G, u, v):
    """Retorna la lista de aristas (como pares) que conforman el camino más corto de u a v."""
    p = shortest_path_nodes(G, u, v)
    return list(zip(p[:-1], p[1:]))

def draw_route(G, positions, path_nodes, start, end):
    plt.figure(figsize=(8, 6))

    # Dibuja grafo base
    nx.draw_networkx_nodes(G, positions, node_size=400)
    nx.draw_networkx_edges(G, positions, arrows=True)
    nx.draw_networkx_labels(G, positions, font_size=8)

    # Construir la secuencia completa de visita start -> ...path_nodes... -> end
    seq = [start] + path_nodes + [end]

    # Unir todos los tramos de caminos más cortos en una lista de aristas
    route_edges = []
    for a, b in zip(seq, seq[1:]):
        route_edges.extend(edges_in_shortest_path(G, a, b))

    # Dibuja las aristas de la ruta con mayor grosor
    nx.draw_networkx_edges(G, positions, edgelist=route_edges, width=3, arrows=True)

    plt.axis("off")
    plt.title("Ruta final (grosor indica tramos utilizados)")
    plt.show()

# Visualizar la ruta optimizada
draw_route(G, positions, opt_path, start, end)



## 6. (Opcional) Held–Karp exacto para pocas ubicaciones

Implementación educativa del algoritmo de **programación dinámica** para TSP Caminante (sin regresar al inicio) con depósitos `start` y `end`.
> Úsalo solo si el número de ítems es **pequeño** (p.ej., ≤ 12).


In [ ]:

def held_karp_path(items, D, start, end):
    # Mapear items a índices (0..n-1)
    n = len(items)
    idx = {items[i]: i for i in range(n)}
    # DP: dict con clave (subset_mask, j) -> costo mínimo para llegar a j visitando 'subset_mask'
    from math import inf
    DP = {}
    parent = {}

    # Inicialización: desde start a cada j individual
    for j in range(n):
        mask = 1 << j
        DP[(mask, j)] = D[start][items[j]]
        parent[(mask, j)] = None

    # Iterar sobre tamaños de subconjunto
    for size in range(2, n+1):
        for subset in itertools.combinations(range(n), size):
            mask = 0
            for j in subset:
                mask |= (1 << j)
            for j in subset:
                best = math.inf
                best_i = None
                prev_mask = mask ^ (1 << j)
                for i in subset:
                    if i == j:
                        continue
                    c = DP.get((prev_mask, i), math.inf) + D[items[i]][items[j]]
                    if c < best:
                        best, best_i = c, i
                DP[(mask, j)] = best
                parent[(mask, j)] = (prev_mask, best_i)

    # Cierre hacia end
    full_mask = (1 << n) - 1
    best = math.inf
    best_j = None
    for j in range(n):
        c = DP[(full_mask, j)] + D[items[j]][end]
        if c < best:
            best, best_j = c, j

    # Reconstrucción de ruta
    path = []
    mask = full_mask
    j = best_j
    while j is not None:
        path.append(items[j])
        prev = parent[(mask, j)]
        if prev is None:
            break
        mask, j = prev
    path.reverse()
    return path, best

# Ejecutar Held-Karp si el tamaño es manejable
if len(items) <= 10:
    hk_path, hk_cost = held_karp_path(items, D, start, end)
    print("Held–Karp ruta:", hk_path)
    print("Held–Karp costo:", hk_cost)
else:
    print("Held–Karp omitido (demasiados ítems para DP exacta).")
